In [23]:
from __future__ import print_function
import pandas as pd
import gspread_pandas
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from tqdm import tqdm
from time import sleep

In [2]:
config=gspread_pandas.conf.get_config(conf_dir='/home/dev-cory/learning_english/credential/',file_name='client_secret_english_crawl.json')
#gspread_pandas.conf.get_config
gspread_pandas.conf.get_config
with open('./credential/learning_english.yaml','r') as ymlfile:
    cfg = yaml.load(ymlfile,Loader=yaml.BaseLoader)
app_id = cfg['OxfordAPI']['app_id']
app_key = cfg['OxfordAPI']['app_key']
language = 'en'

<function gspread_pandas.conf.get_config(conf_dir=None, file_name='google_secret.json')>

In [4]:
spread = gspread_pandas.Spread('Programming experiment of Oxford vocabulary',config=config)

In [5]:
df=spread.sheet_to_df(sheet='vocab')

In [6]:
df

,Word,Importance,Meaning in Japanese,Meaning in English,Example,Origin,phonic,mp3
order,,,,,,,,
1,retaliate,3,仕返しする,1: make an attack in return for a similar atta...,1: the blow stung and she retaliated immediate...,,,
2,detain,3,拘束する,keep (someone) from proceeding by holding them...,"1: she made to open the door, but he detained ...",,,
3,resilient,3,はね返る、弾力のある、たちまち元気を回復する、快活な、溌刺(はつらつ)とした,(of a person or animal) able to withstand or r...,1: babies are generally far more resilient tha...,,,
4,lineage,4,血統、系統、家柄,direct descent from an ancestor; ancestry or p...,1: a Dutch nobleman of ancient lineage 2: the...,,,
5,migration,1,移住、転住、渡り、移住者群、移動する動物の群,1: seasonal movement of animals from one regio...,1: this butterfly's annual migration across No...,,,
...,...,...,...,...,...,...,...,...
1148,thorny,,とげの多い、とげのような、困難な、苦しい,,"This bilateral pact is more limited in scope, ...",,,
1149,fold,,折る、折り重ねる、折りたたむ、折り曲げる、折り返す、収める、組む、組み合わせる、(…に)巻き...,,News that PizzaExpress might fold unless it ca...,,,
1150,ambush,,待ち伏せ、待ち伏せ攻撃、待ち伏せ場所、伏兵、伏せ勢,,We have verified with the Hong Kong Cybersecur...,,,


In [11]:
 df.loc['4','Meaning in Japanese']

'血統、系統、家柄'

In [7]:
def weblio_crawler(word_id):
    url ='https://ejje.weblio.jp/content/'+word_id
    try:
        html_page = urlopen(url)
        soup = BeautifulSoup(html_page)
        text=soup.find('td',attrs={"class", "content-explanation ej"})
        answer=text.contents[0]
    except:
        answer=''
    return answer

In [8]:
def Oxford_definition_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        x=1
        answer=''
        for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
            answer=answer +str(x)+': '+str(i["definitions"][0])+'  '
            x+=1
            sleep(5)
    except:
        answer=''
    return answer

In [31]:
def Oxford_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        sleep(5)
    except Exception as e:
        print(e)
        info=''
    return info

In [32]:
def Oxford_example_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        x=1
        answer=''
        for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
            answer=answer +str(x)+': '+str(i["examples"][0]["text"])+'  '
            x+=1
            sleep(5)
    except:
        answer=''
    return answer

In [33]:
def Oxford_info_processor(info,target):
    try:
        answer=''
        if target=='meaning':
            x=1
            for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
                answer=answer +str(x)+': '+str(i["definitions"][0])+' \r\n'
                x+=1       
        elif target=='phonic':
            answer=info["results"][0]['lexicalEntries'][0]["pronunciations"][0]["phoneticSpelling"]
        elif target=='example':
            x=1
            for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
                answer=answer +str(x)+': '+str(i["examples"][0]["text"])+' \r\n'
                x+=1
        elif target=='mp3':
            answer=info["results"][0]['lexicalEntries'][0]["pronunciations"][0]["audioFile"]
        elif target=='origin':
            answer=info["results"][0]['lexicalEntries'][0]['entries'][0]['etymologies'][0]
    except Exception as e:
        print(e)
        answer=''
    return answer

In [ ]:
for index in tqdm(df.index):
    word_id=df.loc[index,'Word']
    word_id=word_id.strip(' ')
    word_id=word_id.lower()
    if df.loc[index,'Meaning in Japanese']=='':
        df.loc[index,'Meaning in Japanese']=weblio_crawler(word_id)
    if df.loc[index,'Meaning in English']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Meaning in English']=Oxford_info_processor(info,'meaning')
        #print(Oxford_info_processor(info,'meaning'))
        df.loc[index,'Example']=Oxford_info_processor(info,'example')
        df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
        df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
        #print(Oxford_info_processor(info,'phonic'))
        df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    if df.loc[index,'Example']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Example']=Oxford_info_processor(info,'example')
        df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
        df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
        df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    if df.loc[index,'Origin']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
        df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
        df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    
        



  0%|          | 0/1141 [00:00<?, ?it/s]

  0%|          | 4/1141 [00:05<26:29,  1.40s/it]

  0%|          | 5/1141 [00:11<49:51,  2.63s/it]

'etymologies'




  1%|          | 6/1141 [00:16<1:06:26,  3.51s/it]

  1%|          | 7/1141 [00:22<1:17:42,  4.11s/it]

  1%|          | 8/1141 [00:28<1:27:43,  4.65s/it]

'pronunciations'
'pronunciations'




  1%|          | 9/1141 [00:33<1:32:43,  4.91s/it]

  1%|          | 10/1141 [00:39<1:36:30,  5.12s/it]

  1%|          | 11/1141 [00:44<1:39:27,  5.28s/it]

  1%|          | 12/1141 [00:50<1:41:06,  5.37s/it]

  1%|          | 13/1141 [00:56<1:42:14,  5.44s/it]

  1%|          | 14/1141 [01:01<1:43:02,  5.49s/it]

  1%|▏         | 15/1141 [01:07<1:43:27,  5.51s/it]

  1%|▏         | 16/1141 [01:12<1:43:40,  5.53s/it]

  1%|▏         | 17/1141 [01:18<1:43:27,  5.52s/it]

...



 25%|██▍       | 280/1141 [29:18<2:29:49, 10.44s/it]

'etymologies'




 25%|██▍       | 281/1141 [29:24<2:09:01,  9.00s/it]

 25%|██▍       | 282/1141 [29:29<1:54:13,  7.98s/it]

 25%|██▍       | 283/1141 [29:35<1:43:55,  7.27s/it]

 25%|██▍       | 284/1141 [29:41<1:36:58,  6.79s/it]

'pronunciations'
'pronunciations'




 25%|██▍       | 285/1141 [29:46<1:31:33,  6.42s/it]

 25%|██▌       | 286/1141 [29:52<1:27:37,  6.15s/it]

 25%|██▌       | 287/1141 [29:57<1:25:27,  6.00s/it]

 25%|██▌       | 288/1141 [30:03<1:23:47,  5.89s/it]

In [ ]:
spread.df_to_sheet(df, index=True, sheet='result', start='A1', replace=True)

In [ ]:
df